<a href="https://colab.research.google.com/github/reeshabhkumarranjan/ML-Course-Project/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
import numpy as np
import glob
from keras import Sequential
import sklearn, sklearn.model_selection
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, TimeDistributed, Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Activation
from keras.utils import to_categorical

In [2]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# loading data
colab_directory = '/content/drive/My Drive/Colab Notebooks/ML Course Project/'
x = np.load(colab_directory + "x.npy")
y = np.load(colab_directory + "y.npy")
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [16]:
# LSTM model
epochs = 15
model_lstm = Sequential([
	LSTM(200, input_shape=(x_train.shape[1], x_train.shape[2])),
	Dropout(0.5),
	Dense(200),
	Activation('relu'),
	Dense(y_train.shape[1]),
	Activation('softmax'),
])
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(x_train, y_train, epochs=epochs, batch_size=64, verbose=True)
useless, accuracy = model.evaluate(x_test, y_test, batch_size=64, verbose=False)



396/396 [==============================] - 1s 2ms/step - loss: 0.4831 - acc: 0.8914
Epoch 6/15
396/396 [==============================] - 1s 2ms/step - loss: 0.3176 - acc: 0.9293
Epoch 7/15
396/396 [==============================] - 1s 2ms/step - loss: 0.3078 - acc: 0.9040
Epoch 8/15
 64/396 [===>..........................] - ETA: 0s - loss: 0.3012 - acc: 0.8750

KeyboardInterrupt: ignored

In [0]:
# evaluation cnn-lstm

# fit and evaluate a model
def evaluate_model_lstm_cnn(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 1, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape data into time steps of sub-sequences
	n_steps, n_length = 3, 20
	trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
	# define model
	model = Sequential()
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
	model.add(TimeDistributed(Dropout(0.5)))
	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
	model.add(TimeDistributed(Flatten()))
	model.add(LSTM(200))
	model.add(Dropout(0.5))
	model.add(Dense(200, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=True)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=False)
	return accuracy

In [0]:
# LSTM + conv
# fit and evaluate a model
def evaluate_model_lstm_conv(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 1, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape into subsequences (samples, time steps, rows, cols, channels)
	n_steps, n_length = 3, 20
	trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
	# define model
	model = Sequential()
	model.add(ConvLSTM2D(filters=100, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(200, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=True)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=False)
	return accuracy

In [13]:
# running the models
print("Evaluating LSTM model")
accuracy_lstm = evaluate_model(x_train, y_train, x_test, y_test)
print("Accuracy on test:", accuracy_lstm)
print()
print("Evaluating LSTM-CNN model")
accuracy_lstm_cnn = evaluate_model_lstm_cnn(x_train, y_train, x_test, y_test)
print("Accuracy on test:", accuracy_lstm_cnn)
print()
print("Evaluating LSTM-Conv model")
accuracy_lstm_conv = evaluate_model_lstm_conv(x_train, y_train, x_test, y_test)
print("Accuracy on test:", accuracy_lstm_conv)
print()

Evaluating LSTM model
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/15





396/396 [==============================] - 6s 16ms/step - loss: 1.0687 - acc: 0.4242
Epoch 2/15
396/396 [==============================] - 1s 2ms/step - loss: 0.9518 - acc: 0.4798
Epoch 3/15
396/396 [==============================] - 1s 2ms/step - loss: 0.6167 - acc: 0.6944
Epoch 4/15
396/396 [==============================] - 1s 2ms/step - loss: 0.5462 - acc: 0.8258
Epoch 5/15
396/396 [==============================] - 1s 2ms/step - loss: 0.3242 - acc: 0.9167
Epoch 6/15
396/396 [==============================] - 1s 2ms/step - loss: 0.4150 - acc: 0.8460
Epoch 7/15
396/396 [==============================] - 1s 2ms/step - loss: 0.4482 - acc: 0.8359
Epoch 8/15
396/396 [==============================] - 1s 2ms/step - loss: 0.4268 - acc: 0.8510
Epoch 9/15
396/396 [==============================] - 1s 3ms/step - loss: 0.4188 - acc: 0.8636
Epoch 10/15
396/396 

NameError: ignored